本次实验以AAAI 2014会议论文数据为基础，要求实现或调用无监督聚类算法，了解聚类方法。

### 任务介绍
每年国际上召开的大大小小学术会议不计其数，发表了非常多的论文。在计算机领域的一些大型学术会议上，一次就可以发表涉及各个方向的几百篇论文。按论文的主题、内容进行聚类，有助于人们高效地查找和获得所需要的论文。本案例数据来源于AAAI 2014上发表的约400篇文章，由[UCI](https://archive.ics.uci.edu/ml/datasets/AAAI+2014+Accepted+Papers!)公开提供，提供包括标题、作者、关键词、摘要在内的信息，希望大家能根据这些信息，合理地构造特征向量来表示这些论文，并设计实现或调用聚类算法对论文进行聚类。最后也可以对聚类结果进行观察，看每一类都是什么样的论文，是否有一些主题。

基本要求：
1. 将文本转化为向量，实现或调用无监督聚类算法，对论文聚类，例如10类（可使用已有工具包例如sklearn）；
2. 观察每一类中的论文，调整算法使结果较为合理；
3. 无监督聚类没有标签，效果较难评价，因此没有硬性指标，跑通即可，主要让大家了解和感受聚类算法，比较简单。

扩展要求：
1. 对文本向量进行降维，并将聚类结果可视化成散点图。

注：group和topic也不能完全算是标签，因为
1. 有些文章作者投稿时可能会选择某个group/topic但实际和另外group/topic也相关甚至更相关；
2. 一篇文章可能有多个group和topic，作为标签会出现有的文章同属多个类别，这里暂不考虑这样的聚类；
3. group和topic的取值很多，但聚类常常希望指定聚合成出例如5/10/20类；
4. 感兴趣但同学可以思考利用group和topic信息来量化评价无监督聚类结果，不作要求。

提示：
1. 高维向量的降维旨在去除一些高相关性的特征维度，保留最有用的信息，用更低维的向量表示高维数据，常用的方法有PCA和t-SNE等；
2. 降维与聚类是两件不同的事情，聚类实际上在降维前的高维向量和降维后的低维向量上都可以进行，结果也可能截然不同；
3. 高维向量做聚类，降维可视化后若有同一类的点不在一起，是正常的。在高维空间中它们可能是在一起的，降维后损失了一些信息。

输入数据

In [6]:
import pandas as pd
def load_data(path='./data/[UCI]AAAI-14AcceptedPapers-Papers.csv'):
    df_data = pd.read_csv(path)
    df_data = df_data.dropna()
    return df_data

df = load_data()
df.describe() 

,title,authors,groups,keywords,topics,abstract
count,392,392,392,392,392,392
unique,392,387,125,392,348,392
top,Kernelized Bayesian Transfer Learning,Chen Chen and Vincent Ng,Novel Machine Learning Algorithms (NMLA),cross-domain learning\ndomain adaptation\nkern...,GTEP: Social Choice / Voting,Transfer learning considers related but distin...
freq,1,2,49,1,7,1


去除不客观的特征（groups & topics）

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import numpy as np

df_feature = df.drop(['groups', 'topics'],axis=1)

cv = CountVectorizer(min_df=0.0, max_df=1.0)
X_fea = [[i] for i in range(df.shape[0])]
for col in df_feature.columns:
    tmp = cv.fit_transform(df_feature[col])
    # 每个feature是【样本数×词袋大小】的
    feature = tmp.toarray()
    X_fea = np.concatenate((X_fea, feature), axis=1)

X_fea.shape

(392, 9899)

# 取出最能代表文章主题的三个特征作为后面的评价指标

In [8]:

from sklearn.metrics import silhouette_score, calinski_harabasz_score    
df_cls = df[['keywords', 'groups', 'topics']]
cv = CountVectorizer(min_df=0.2, max_df=1.0)
X_cls = [[i] for i in range(df.shape[0])]
for col in df_cls.columns:
    tmp = cv.fit_transform(df_cls[col])
    feature = tmp.toarray()
    X_cls = np.concatenate((X_cls, feature), axis=1)

def cal_score(result_list):
    global X_cls
    sh = silhouette_score(X_cls, result_list)
    ch = calinski_harabasz_score(X_cls, result_list)
    return sh,ch

In [11]:
from sklearn.cluster import KMeans    
from sklearn.decomposition import PCA
import collections
rec = collections.defaultdict(list)
for _cmp in [2, 5, 10, 50, 100]:
    X_pca = PCA(n_components=_cmp).fit_transform(X_fea)
    print(X_pca.shape)
    for _k in range(5, 16):
        model = KMeans(n_clusters=_k)
        res = model.fit_predict(X_pca)
        sh,ch = cal_score(res)
        print('k:',_k, 'sh:', sh, 'ch:', ch, 'score:', sh*ch)
        rec[_cmp].append(sh*ch)

(392, 2)


NameError: name 'silhouette_score' is not defined